In [23]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [53]:
import time
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import bs4
from bs4 import BeautifulSoup 
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

In [54]:
driver=webdriver.Chrome(r"chromedriver.exe")
driver.get("https://www.fotocasa.es")
driver.maximize_windows()

AttributeError: 'WebDriver' object has no attribute 'maximize_windows'

In [55]:
time.sleep(2)
alquilar=driver.find_element_by_xpath('.//div[@class="re-HomeSearchSelector-item re-HomeSearchSelector-item--rent"]')
alquilar.click()

In [56]:
time.sleep(2)
buscador=driver.find_element_by_xpath('.//div[@class="sui-MoleculeAutosuggest-input-container"]/input')
buscador.click()


In [57]:
buscador.send_keys('Bilbao')
time.sleep(1)
buscador.send_keys(Keys.ENTER)

In [304]:
for i in range(20):
    html_txt25=driver.page_source
    soup=BeautifulSoup(html_txt25)
    listaprecios=[]

    precios = soup.find_all('div', class_="re-CardPriceComposite")
    for precio in precios:
        pr=precio.find('span').getText()
        listaprecios.append(pr)
    
    ActionChains(driver).key_down(Keys.PAGE_DOWN).key_up(Keys.PAGE_DOWN).perform()
    time.sleep(0.5)
    
listahabitaciones=[]
    
habitaciones = soup.find_all('span', class_="re-CardFeatures-feature") 

for habitacion in habitaciones:
    hab=habitacion.getText()
    listahabitaciones.append(hab) 
    

listatipo=[]
    
#tipos = soup.find_all('div', class_="re-Card-wrapperTitle") 
tipos = soup.find_all('span', class_="re-CardTitle re-CardTitle--ayce") 
for tipo in tipos:
    tip=tipo.find('span').getText()
    listatipo.append(tip)
        

listacalles=[]
    
#calles = soup.find_all('div', class_="re-Card-wrapperTitle") 
calles = soup.find_all('span', class_="re-CardTitle re-CardTitle--ayce") 
for calle in calles:
#    cal=calle.find('h3').getText()
    cal=calle.getText()
    if "Piso" in cal:
        cal=cal[4:]
        listacalles.append(cal)
    else:
        if "Apartamento" in cal:
            cal=cal[11:]
            listacalles.append(cal)
        else:
            if "Ático" in cal:
                cal=cal[5:]
                listacalles.append(cal)
            else:
                if "Dúplex" in cal:
                    cal=cal[6:]
                    listacalles.append(cal)
                else:
                    if "Estudio" in cal:
                        cal=cal[7:]
                        listacalles.append(cal)
                    else:
                        listacalles.append(cal)   
                            


In [305]:
print(listahabitaciones)
print(listaprecios)
print(listatipo)
print(listacalles)

['5 habs.', '3 baños', '145 m²', '', '3 habs.', '2 baños', '100 m²', '', '100 m²', '', '4 habs.', '112 m²', '', '4 habs.', '110 m²', '', '4 habs.', '109 m²', '', '4 habs.', '4 baños', '158 m²', '']
['1.900 € /mes', '1.000 € /mes', '1.100 € /mes', '1.800 €', '1.800 €', '1.800 €', 'A consultar']
['Piso', 'Piso', 'Piso']
['Deusto', 'Deusto', 'Buenos Aires, Abando - Albia']


In [306]:
ind_comienzo = 0
ind_reg_nuevo = 0

# Indicadores para que todas las listas tengan el mismo número de elementos.
ind_precio = 0
ind_hab = 0
ind_habs = 0
ind_bano = 0
ind_banos = 0
ind_metros = 0
ind_ascensor = 0
ind_terraza = 0
ind_parking = 0
col_ignorar = []
ignorar = 0
contador = 0
num_ignorados = 0

#Lista de elementos para comentar las columnas
col_precios=[]
col_habitaciones=[]
col_banos=[]
col_metros=[]
col_terraza=[]
col_ascensor=[]
col_parking=[]

for elemento in listahabitaciones:
# Habitaciones
    if "habs" in elemento:
        ind_habs = 1
        ind_reg_nuevo = 1
        hab = elemento[-7]
        col_habitaciones.append(hab) 
        
    else:
        if "hab" in elemento:
            ind_hab = 1
            ind_reg_nuevo = 1
            hab = elemento[-6]
            col_habitaciones.append(hab) 
# Baños            
    if "baños" in elemento:
        if ind_reg_nuevo == 0 and (ind_bano == 1 or ind_banos ==1):
            ignorar = 1
        else:
            ind_banos = 1
            ban = elemento[-7]
            col_banos.append(ban)       
    else:
        if "baño" in elemento:           
            if ind_reg_nuevo == 0 and (ind_bano == 1 or ind_banos ==1):
                ignorar = 1
            else:
                ind_bano = 1
                ban = elemento[-6]
                col_banos.append(ban)      
# Metros cuadrados
    if "m²" in elemento:
        if ind_reg_nuevo == 0 and ind_metros == 1:
             ignorar = 1
        else:
            ind_metros = 1
            met = elemento[0:-3]
            col_metros.append(met)   
# Ascensor
    if "ascensor" in elemento:
        if ind_reg_nuevo == 0 and ind_ascensor == 1:
             ignorar = 1
        else:
            ind_ascensor = 1
            col_ascensor.append('S')  
# Terraza
    if "terraza" in elemento:
        ind_terraza = 1
        col_terraza.append('S') 
# Parking
    if "parking" in elemento:
        ind_parking = 1
        col_parking.append('S') 
        
    if ind_comienzo == 0:  
        ind_comienzo = 1
        ind_reg_nuevo = 0
    else:
# Si es un registro nuevo tenemos que escribir a ceros o a espacios aquellos datos de los que no tengamos información
        if ind_reg_nuevo == 1:
            if  ind_bano == 0 and ind_banos == 0:
                col_banos.append(0) 
            if ind_metros == 0:
                col_metros.append(0)
            if ind_terraza == 0:
                col_terraza.append('N')      
            if ind_ascensor == 0:
                col_ascensor.append('N')    
            if ind_parking == 0:
                col_parking.append('N')   
 # Inicializamos contadores.           
            ind_reg_nuevo = 0
            ind_habs = 0
            ind_hab = 0
            ind_bano = 0
            ind_banos = 0
            ind_metros = 0
            ind_terraza = 0
            ind_ascensor = 0
            ind_parking = 0

            if  ignorar == 1:
                col_ignorar.append(contador)
                contador = contador + 1 
                num_ignorados = num_ignorados + 1
                
            ignorar = 0
            contador = contador + 1 
# Cuidado con el último, porque solo escribiremos la habitación  
if  ind_bano == 0 and ind_banos == 0:
    col_banos.append(0) 
if ind_metros == 0:
    col_metros.append(0)
if ind_terraza == 0:
    col_terraza.append('N')      
if ind_ascensor == 0:
    col_ascensor.append('N')    
if ind_parking == 0:
    col_parking.append('N') 

# Miramos como vamos

print(col_habitaciones)          
print(col_banos)
print(col_metros)
print(col_terraza)
print(col_ascensor)
print(col_parking)


['5', '3', '4', '4', '4', '4']
['3', '2', 0, 0, 0, '4']
['145', '100', '112', '110', '109', '158']
['N', 'N', 'N', 'N', 'N', 'N']
['N', 'N', 'N', 'N', 'N', 'N']
['N', 'N', 'N', 'N', 'N', 'N']


In [307]:
col_ignorar

[1]

In [308]:
print(num_ignorados)

1


In [309]:
# Extraemos los precios
col_precios=[]
contador = 0
existe_lista = 0

for elemento in listaprecios:
    if "€ /mes" in elemento:
        pre = elemento[0:-7]
        for lista in col_ignorar:           
            if contador == lista:
                existe_lista = 1
                
        if existe_lista == 0:
            pre=pre.replace('.', '')
            col_precios.append(pre)      
    else:
        col_precios.append(0) 
        
    contador = contador + 1
    existe_lista = 0

print(col_precios)

['1900', '1100', 0, 0, 0, 0]


In [310]:
#Ignorar también los tipos y las calles de los registros que vayamos ignorando.
# Extraemos los tipos
col_tipos=[]
contador = 0
existe_lista = 0
 
for elemento in listatipo:
    tipo = elemento
    for lista in col_ignorar:           
        if contador == lista:
            existe_lista = 1
    if existe_lista == 0:
        col_tipos.append(tipo)       
   
    contador = contador + 1
    existe_lista = 0

print(col_tipos)

# Extraemos los tipos
col_calles=[]
contador = 0
existe_lista = 0

for elemento in listacalles:
    calle = elemento
    for lista in col_ignorar:           
        if contador == lista:
            existe_lista = 1
    if existe_lista == 0:
        col_calles.append(calle)    
            
    contador = contador + 1
    existe_lista = 0

print(col_calles)

['Piso', 'Piso']
['Deusto', 'Buenos Aires, Abando - Albia']


In [311]:
df=pd.DataFrame()

In [312]:
df=pd.DataFrame()
df=pd.DataFrame({'Ciudad': 'Bilbao', 'Habitaciones': col_habitaciones, 'Baños':col_banos,  'Superficie':col_metros, 
                 'Terraza': col_terraza, 'Ascensor': col_ascensor, 'Parking': col_parking, 'Tipo_vivienda': col_tipos,
                  'Calle': col_calles, 'Precio': col_precios 
                })

ValueError: arrays must all be same length

In [ ]:
#print(df)

In [ ]:
df.to_csv('PRECIOS_bis_25.CSV', index=False)